# The Hamiltonian

In [ ]:
import numpy as np
from math import *
from numpy import sin, cos

# Constants
a = 3.905
tl = 875 *10**(-3)/2.
th = 40 *10**(-3)/2.
td = 40 *10**(-3)/2.
U = 2.7
dE =  47*10**(-3)
dso = 40*10**(-3)

XYD, XYU, XZD, XZU, YZD, YZU = 0, 1, 2, 3, 4, 5
REAL,IMAG = 1, 2
BOTH= REAL | IMAG

N1_ELEM=(XYD, XYD, REAL)
N2_ELEM=(XZD, XZD, REAL)
Na_ELEM=(XYD, XZU, IMAG)
Nb_ELEM=(XZD, YZD, IMAG)

HARTREE_ELEMS = [N1_ELEM, N2_ELEM]
FOCK_ELEMS = [N1_ELEM, N2_ELEM, Na_ELEM, Nb_ELEM]

def create_H(kxa, kya, N1, N2, N3):
    
    e1 = 2*tl*(2 - cos(kxa) - cos(kya))
    e2 = 2*tl*(1 - cos(kxa)) + 2*th*(1 - cos(kya))
    e3 = 2*th*(1 - cos(kxa)) + 2*tl*(1 - cos(kya))
    
    ### H0 Matrix
    H0_little = np.matrix([[e1-dE,           0,                        0               ],
                           [  0,             e2,               2*td*sin(kxa)*sin(kya)  ],
                           [  0,     2*td*sin(kxa)*sin(kya),           e3              ]])
    
    H0 = np.kron(H0_little, np.identity(2))
        
    ### Hso Matrix
    Hso = (1j*dso/2)*np.matrix([[ 0,  0,  0,  1,  0, 1j],
                                [ 0,  0,  1,  0,-1j,  0], 
                                [ 0, -1,  0,  0,  1,  0], 
                                [-1,  0,  0,  0,  0, -1], 
                                [ 0,-1j, -1,  0,  0,  0], 
                                [1j,  0,  0,  1,  0,  0]])
    
    ### Hint matrix
    E1 = 0.5*U*N1 +     U*N2 +     U*N3;
    E2 =     U*N1 + 0.5*U*N2 +     U*N3;
    E3 =     U*N1 +     U*N2 + 0.5*U*N3;
    
    Hint_little = np.matrix([[E1, 0,   0], 
                             [0,  E2,  0],
                             [0,   0, E3]])
    
    Hint = np.kron(Hint_little, np.identity(2))
    
    return H0 + Hso + Hint 

#import numba
#@numba.jit(nopython=1)
def create_full_H_faster(kxa, kya, N):
    
    # normal energies
    e1 = 2*tl*(2 - cos(kxa) - cos(kya))
    e2 = 2*tl*(1 - cos(kxa)) + 2*th*(1 - cos(kya))
    e3 = 2*th*(1 - cos(kxa)) + 2*tl*(1 - cos(kya))    
    es = 2*td*sin(kxa)*sin(kya)
    
    ### Interaction erergies
    E1 =   U*N[0] + 2*U*N[1] + 2*U*N[1];
    E2 = 2*U*N[0] +   U*N[1] + 2*U*N[1];
    E3 = 2*U*N[0] + 2*U*N[1] +   U*N[1];
    Na = 1j*N[2]
    Nb = 1j*N[3]
    
    ### The whole H Matrix
    #H = np.zeros((6,6), dtype=numba.complex128)
    H = np.zeros((6,6), dtype=np.complex128)
    H[0,0]=e1-dE+E1    ;H[0,1]=    0       ;H[0,2]=       0       ;H[0,3]=  1j*dso/2-Na ;H[0,4]=     0        ;H[0,5]=-dso/2+1j*Na
    H[1,0]=   0        ;H[1,1]=e1-dE + E1  ;H[1,2]=   1j*dso/2-Na ;H[1,3]=      0       ;H[1,4]= dso/2-1j*Na  ;H[1,5]=      0    
    H[2,0]=   0        ;H[2,1]=-1j*dso/2+Na;H[2,2]=    e2 + E2    ;H[2,3]=      0       ;H[2,4]=1j*dso/2+es-Nb;H[2,5]=      0
    H[3,0]=-1j*dso/2+Na;H[3,1]=    0       ;H[3,2]=       0       ;H[3,3]=   e2 + E2    ;H[3,4]=     0        ;H[3,5]=-1j*dso/2+es+Nb
    H[4,0]=   0        ;H[4,1]=dso/2-1j*Na ;H[4,2]=-1j*dso/2+es+Nb;H[4,3]=      0       ;H[4,4]=  e3 + E3     ;H[4,5]=      0
    H[5,0]=-dso/2+1j*Na;H[5,1]=    0       ;H[5,2]=       0       ;H[5,3]=1j*dso/2+es-Nb;H[5,4]=     0        ;H[5,5]=   e3 + E3
        
    return H

class InlineH:
    def __init__(self, N):
        self.H = create_full_H_faster(0, 0, N)
        self.E1 =   U*N[0] + 2*U*N[1] + 2*U*N[1];
        self.E2 = 2*U*N[0] +   U*N[1] + 2*U*N[1];
        self.E3 = 2*U*N[0] + 2*U*N[1] +   U*N[1];
        self.Nb = 1j*N[3]
            
    def update_k(self, kxa, kya):
        e1 = 2*tl*(2 - cos(kxa) - cos(kya))
        e2 = 2*tl*(1 - cos(kxa)) + 2*th*(1 - cos(kya))
        e3 = 2*th*(1 - cos(kxa)) + 2*tl*(1 - cos(kya))    
        es = 2*td*sin(kxa)*sin(kya)
        
        self.H[0,0] = self.H[1,1] = e1 - dE + self.E1;
        self.H[2,2] = self.H[3,3] = e2 + self.E2
        self.H[4,4] = self.H[5,5] = e3 + self.E3
        self.H[4,2] = self.H[3,5] = -1j*dso/2 + es + self.Nb
        self.H[5,3] = self.H[2,4] =  1j*dso/2 + es - self.Nb

# Fock Elements

In [ ]:
from scipy import integrate, optimize
from sympy.functions.special.delta_functions import Heaviside

integrand_counter = 0
energy_counter = 0
bound_counter = 0
statistics = []

def zero_statistics():
    global integrand_counter, energy_counter, bound_counter, statistics
    integrand_counter, energy_counter, bound_counter = 0, 0, 0
    statistics = []

### element: [ (i,j, real/imag) ... ]
def fock_element_integrand(N, mu, elements):
    def fock_elem_integrand(kxa, kya, ms):
        global integrand_counter, statistics
        integrand_counter += 1
        statistics.append((kxa, kya))
        H = create_full_H_faster(kxa, kya, N) 
        e, psi = np.linalg.eigh(H)
        
        # leave only the cols asked asked by m
        cols = np.array([(int(i/2) in ms) for i in range(6)])    
        psi = psi[:,cols]
                
        total = np.array([
                np.sum(
                    np.multiply(
                        psi[elm[0],:], 
                        np.conj(psi[elm[1],:])
                    )  # multiplies the elements of psi[elm[0],:] and 
                       # psi[elm[1],:]* elementwise
                ) # sum over all the elements - actually, over all the m in ms

            for elm in elements
        ]) # for each element requested (for optimization, it is faster to
           # calculate several elemets at once)
        
        ans = np.array([])
        # seperate the real and imag
        for i, elem in enumerate(elements):
            if elem[2]&REAL != 0:
                ans = np.append(ans, np.array([np.real(total[i])]))
            if elem[2]&IMAG != 0:
                ans = np.append(ans, np.array([np.imag(total[i])]))
            
        return  (1/(2*pi)**2) * ans
            
    return fock_elem_integrand

def m_energy(N, mu):
    def energy(kxa, kya, ms):
        global energy_counter
        energy_counter += 1
        H = create_full_H_faster(kxa, kya, N) 
        e = np.linalg.eigvalsh(H)
        return  [e[2*m] for m in ms]
    return energy

# this function assumes that the eig_values are monotoneous .
# @return for every kxa, the positive kya value that will match the fermi area
def m_fermi_area(N, m, mu):
    def area(kxa):
        global bound_counter
        bound_counter += 1
        
        e_func = m_energy(N, mu)
        fermi_func = lambda kya: e_func(kxa, kya, [m])[0] - mu 
        
        fa, fb = fermi_func(0), fermi_func(pi)
        
        # find the first kya value of eig_func that is above mu
        if np.sign(fa) == -1 and np.sign(fb) == -1:
            return pi
        elif np.sign(fa) == 1 and np.sign(fb) == 1:
            return 0
        else:
            return optimize.brentq(fermi_func, 0, pi)
    return area

integ = fock_element_integrand([1/2, 1/2, 0, 0], 0.95, [(1, 1, REAL)])
integ(0, 0, [0])

In [ ]:
def integrate_over_fermi(N, mu, f):   
    # integrate over kxa, kya in the fermi area for every m
    total = 0
    
    total += integrate.dblquad(
        lambda y, x: f(x, y, 0), 0, pi, lambda x: 0, m_fermi_area(N, 0, mu),
    )[0]
    total += integrate.dblquad(
        lambda y, x: f(x, y, 1), 0, pi, lambda x: 0, m_fermi_area(N, 1, mu),
    )[0]
    total += integrate.dblquad(
        lambda y, x: f(x, y, 2), 0, pi, lambda x: 0, m_fermi_area(N, 2, mu),
    )[0]
    
    if abs(total) < 0.1e-8:
        total = 0
        
    return 4*total

def fock_element(N, mu, element):   
    f = lambda kxa, kya, i: fock_element_integrand(N, mu, [element])(kxa, kya, [i])[0]
    return integrate_over_fermi(N, mu, f)

def fock_elements_slower(N, mu, elements):
    ans = []
    for elem in elements:
        ans += [fock_element(N, mu, elem)]
    return np.array(ans)

def fock_elements_faster(N, mu, elements):
    
    integrand = fock_element_integrand(N[0], N[1], mu, elements)
    area_1 = m_fermi_area(N, 2, mu)
    area_2 = m_fermi_area(N, 1, mu)
    area_3 = m_fermi_area(N, 0, mu)

    # first, get a rough assumption of the total area
    rough = 0
    dx = pi/300
    for i in range(300):
        rough += area_3(i*dx)*dx
        
    # now, use it to calculate the dx and dy
    NUM_CALLS = 20000
    dx = dy = sqrt(rough/NUM_CALLS)
    
    # integrate     
    total = np.array([0]*len(elements))
    for i in range(int(pi/dx)):
        x = i*dx
        y_1, y_2, y_3 = area_1(x), area_2(x), area_3(x)
        if y_3 == 0:
            break
        for j in range(int(y_1/dy)):
            y = 0 + j*dy
            total = total + integrand(x, y, [0,1,2])*dx*dy
            y += dy
        for j in range(int((y_2-y_1)/dy)):
            y = y_1 + j*dy
            total = total + integrand(x, y, [0,1])*dx*dy
            y += dy
        for j in range(int((y_3-y_2)/dy)):
            y = y_2 + j*dy
            total = total + integrand(x, y, [0])*dx*dy
            y += dy
            
        x += dx    
        
    return total*4


all_possibilities = [(i,j, REAL) for i in range(6) for j in range(6)]
hartree_elem = [(0, 0, REAL), (2, 2, REAL), (4, 4, REAL)]
mu=0.14
guess_N1, guess_N2 = 0.01070613043,  0.009271217 
#for element in all_possibilities:
#    print element, fock_element([guess_N1, guess_N2, guess_N2], mu, (element))

zero_statistics()
print fock_elements_slower([guess_N1, guess_N2, 0, 0], mu, hartree_elem)
print "The integrand was called", integrand_counter, "times"

#zero_statistics()
#print fock_elements_faster([guess_N1, guess_N2], mu, hartree_elem)
#print "The integrand was called", integrand_counter, "times"

In [ ]:
import cProfile
import pstats
#s = cProfile.run("m_integrate([guess_N1, guess_N2, guess_N3], mu)", "prfiler_data")
#pstats.Stats("prfiler_data").sort_stats("tottime").print_stats()
#%prun m_integrate([guess_N1, guess_N2, guess_N3], mu)

# Cache

In [ ]:
import functools

def global_params_str():
    global a,tl,th,td,U,dE,dso
    return "a=%f,tl=%f,th=%f,td=%f,U=%f,dE=%f,dso=%f" % (a,tl,th,td,U,dE,dso)

# note that this decorator ignores **kwargs
def memoize(obj):
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        
        # first, find the validate that there is a cache for that func
        f = obj.func_name
        if f not in cache:
            cache[f] = {}
        
        # first, for every float arg, round it
        fixed_args = []
        for arg in args: 
            if type(arg) == float:
                arg = round(arg, 3)
            fixed_args.append(arg)
        args = fixed_args
        
        print "Cache: ", str(args)
        g = global_params_str()
        if g not in cache[f]:
            print "Globals not in cache. Creating new one"
            cache[f][g] = {}
        if str(args) not in cache[f][g]:
            print "Not in cache", cache[f][g]
            cache[f][g][str(args)] = obj(*args, **kwargs)
        return cache[f][g][str(args)]
    return memoizer


In [ ]:
# load cache
import pickle
cache = pickle.load(open("new_database_fock", "r"))

In [ ]:
# save cache
import pickle
f = open("new_database_fock", "w")
pickle.dump(cache, f)
f.close()

# Self Consistent Solver

In [ ]:
mu = 0.26

def hartree_zero_me_mu(mu):
    def zero_me(N_in):
        N_in_full = np.append(N_in, [0, 0])
        N_out = fock_elements_slower(N_in_full, mu, HARTREE_ELEMS)[0:2]
        print N_in, "->", N_out
        return N_in - N_out
    return zero_me

def fock_zero_me_mu(mu):
    def zero_me(N_in):
        N_out = fock_elements_slower(N_in, mu, FOCK_ELEMS)
        print N_in, "->", N_out
        return N_in - N_out
    return zero_me

@memoize
def calc_hartree_N(mu):
    global guess_N1, guess_N2
    hartree_N = optimize.fsolve(hartree_zero_me_mu(mu), [guess_N1, guess_N2], xtol=0.01, factor=100)
    guess_N1, guess_N2, = hartree_N
    return np.append(hartree_N, [0, 0])

@memoize
def calc_fock_N(mu):
    global guess_N1, guess_N2, guess_Na, guess_Nb
    fock_N = optimize.fsolve(fock_zero_me_mu(mu), [guess_N1, guess_N2, guess_Na, guess_Nb], 
                                xtol=0.01, factor=100)
    guess_N1, guess_N2, guess_Na, guess_Nb = fock_N
    return fock_N

def calc_n(mu, N):
    ans = np.array([
        integrate.dblquad(
            lambda x, y: 1,
            0.0, pi, lambda x: 0.0, m_fermi_area(N, m, mu),        
        )[0]
        for m in range(3)
    ])
    return 4*ans/((2*pi)**2) / (a*10**(-8))**2

class Solver(object):
    def __init__(self, U, dso, fock):
        self.U, self.dso = U, dso
        self.fock = fock
    
    def set_u_dso(self):
        global U, dso
        U, dso = self.U, self.dso
    
    def calc_N(self, mu):
        zero_statistics()
        self.set_u_dso()
        if self.fock:
            return calc_fock_N(mu)
        else:
            return calc_hartree_N(mu)    
    
    def calc_n(self, mu):
        return calc_n(mu, self.calc_N(mu))
    
    def __str__(self):
        return "%s U=%.1f dso=%.3f" % (["Hartree", "Fock"][int(self.fock)], self.U, self.dso)


guess_Na, guess_Nb = 0, 0
mu = 0.192
zero_statistics()
#fock_zero_me_mu(mu)([guess_N1, guess_N2, guess_Na, guess_Nb])
new_N = calc_fock_N(mu)
print new_N
N = [guess_N1, guess_N2, guess_Na, guess_Nb] = new_N
print "N is", N
print "n is", calc_n(mu, N)

# Map Non Monotoneous Area

In [ ]:
import copy

class uncert_var(object):
    def __init__(self, val, pos, uncert, uncert_matrix):
        self.val = val
        self.pos = pos
        self.uncert = uncert
        self.uncert_matrix = uncert_matrix
    
    def set_val(self, val):
        self.val = val
        self.uncert = np.array([0, 0])
    
    def walk(self, delta):
        self.pos = np.add(self.pos, delta)
        self.uncert = np.add(self.uncert, np.abs(np.dot(self.uncert_matrix, delta)))
    
    def __str__(self):
        return str(self.val) + "+-" + str(self.uncert) + " at " + str(self.pos)
    
delta_mu = 0.04
def find_non_mon(mu, fock):
    print "Trying mu", mu
    solver = Solver(mu.pos[0], mu.pos[1], fock)
    func = lambda mu: solver.calc_n(mu)[1]
    
    def check_deriv(mu):
        mu = int(mu/delta_mu) * delta_mu
        return -(func(mu) - func(mu+delta_mu))
    
    if check_deriv(mu.val) < 0:
        return mu.val
    
    dmu = 0.04
    while dmu < mu.uncert[1] or -dmu > -mu.uncert[0]:
        zero_statistics()

        if dmu < mu.uncert[1]:
            if check_deriv(mu.val + dmu) < 0:
                return mu.val + dmu
            
        if -dmu > -mu.uncert[0]:
            if check_deriv(mu.val - dmu) < 0:
                return mu.val - dmu
            
        dmu += delta_mu
    return False
    
delta_U = [-0.1, 0]
delta_dso = [0.1, -2*10**(-3)]

mu_uncert_mat = np.array([[0.041/0.1, 0.04/0.002], 
                          [0.041/0.1, 0.04/0.002]])

def map_nonmon_area(fock, first_non_mon, last_dso):
    mu_uncert = [0.2, 0.2]
    pos = first_non_mon
    mu = uncert_var(0.12, pos, mu_uncert, mu_uncert_mat)
    history = []

    #dso loop
    while mu.pos[1] > last_dso:
        new_mu_val = find_non_mon(mu, fock)
        history += [(copy.copy(mu), new_mu_val == False)]
        if new_mu_val == False:
            print "Monotoneous in range"
            break

        # U loop
        while new_mu_val != False:
            mu.set_val(new_mu_val)
            mu.walk(delta_U)
            new_mu_val = find_non_mon(mu, fock)
            history += [(copy.copy(mu), new_mu_val == False)]

        print "next dso!", mu
        mu.walk(delta_dso)
    return history

fock_area_map = map_nonmon_area(1, (2.7, 40*10**(-3)), 2*10**(-3))
hartree_area_map = map_nonmon_area(0, (2.7, 40*10**(-3)), 2*10**(-3))

# Conductivity

In [ ]:
# Naive Hall Coefficient
mobility = np.array([6000, 500, 500])

def naive_hall_coef(mu):
    solver = Solver(2.7, 0.04, True)
    n = solver.calc_n(mu)
    return np.square(np.dot(n, mobility)) / np.dot(n, np.square(mobility)), n[1]

mus = [0.02*i for i in range(20)]
naive_halls = [naive_hall_coef(mu) for mu in mus]
pyplot.plot(mus, [nh[0] for nh in naive_halls])
pyplot.plot(mus, [nh[1] for nh in naive_halls])
pyplot.show()

In [ ]:
dk = 0.001

xi_xy = 6000
xi_xz = xi_yz = 500
m_l = 0.7
m_h = 15

m_xy = m_l
m_xz = m_yz = 2 * (m_l * m_h) / (m_l + m_h)

tau_xy = xi_xy*m_xy
tau_xz = xi_xz*m_xz
tau_yz = xi_xz*m_xz

base_tau = np.array([tau_xy, tau_xy, tau_xz, tau_xz, tau_yz, tau_yz])

def m_inline_energy(IH, mu):
    def energy(kxa, kya, ms):
        IH.update_k(kxa, kya)
        e = np.linalg.eigvalsh(IH.H)
        return  [e[2*m] for m in ms]
    return energy

def deriv(func, delta):
    def ans(x):
        return (func(x+delta/2) - func(x-delta/2)) / delta
    return ans

def partial_x(func, delta):
    def ans(x, y):
        return (func(x+delta/2, y) - func(x-delta/2, y)) / delta
    return ans

def partial_y(func, delta):
    def ans(x, y):
        return (func(x, y+delta/2) - func(x, y-delta/2)) / delta
    return ans

def sigma_xx_integrand(N, mu):
    IH = InlineH(N)
    energies = lambda kx, ky: np.array(m_inline_energy(IH, mu)(kx, ky, [0, 1, 2]))[[0, 0, 1, 1, 2, 2]]
    denergies_dkx = partial_x(energies, dk)
    
    def integrand(kxa, kya):
        statistics.append((kxa, kya))
        global integrand_counter
        integrand_counter += 1
        
        IH.update_k(kxa, kya)
        _, psi = np.linalg.eigh(IH.H)
                
        # setting tau[i] = squared_abs(psi[s, i])*base_tau[s] (sum over s)
        tau = np.dot(base_tau, np.square(np.abs(psi)))
                
        # x[i] = tau[i] * square(dx_energies[i])
        x = np.multiply(
            np.square(denergies_dkx(kxa, kya)), 
            tau
        )
        
        return x
        
    return integrand

def sigma_xy_integrand(N, mu):
    IH = InlineH(N)
    energies = lambda kx, ky: np.array(m_inline_energy(IH, mu)(kx, ky, [0, 1, 2]))[[0, 0, 1, 1, 2, 2]]
    denergies_dkx = partial_x(energies, dk)
    denergies_dky = partial_y(energies, dk)
    denergies_dkx_dky = partial_x(partial_y(energies, dk), dk)
    denergies_dky_dky = partial_y(partial_y(energies, dk), dk)
    
    def integrand(kxa, kya):
        statistics.append((kxa, kya))
        global integrand_counter
        integrand_counter += 1
        
        IH.update_k(kxa, kya)
        _, psi = np.linalg.eigh(IH.H)

        # setting tau[i] = squared_abs(psi[s, i])*base_tau[s] (sum over s)
        squared_tau = np.dot(np.square(base_tau), np.square(np.abs(psi)))
        
        # setting delta
        delta = np.multiply(denergies_dky(kxa, kya), denergies_dkx_dky(kxa, kya)) - \
                    np.dot(denergies_dkx(kxa, kya), denergies_dky_dky(kxa, kya))
                
        # x[i] = squared_tau[i] * square(dx_energies[i]) * delta
        x = np.multiply(
            np.multiply(
                denergies_dkx(kxa, kya),
                delta
            ),
            squared_tau
        )
        
        return x
        
    return integrand
mu = 0.2
N = calc_fock_N(mu)
integrand = sigma_xx_integrand(N, mu)
hall_integrand = sigma_xy_integrand(N, mu)
print "normal %s, hall %s" % (integrand(1, 1), hall_integrand(1, 1))

In [ ]:
@memoize
def sigma_xx(mu):
    N = calc_fock_N(mu)
    xx_integrand = sigma_xx_integrand(N, mu)
    f = lambda kx, ky, i: xx_integrand(kx, ky)[i*2]
    return integrate_over_fermi(N, mu, f)

@memoize
def sigma_xy(mu):
    N = calc_fock_N(mu)
    xy_integrand = sigma_xy_integrand(N, mu)
    f = lambda kx, ky, i: xy_integrand(kx, ky)[i*2]
    return integrate_over_fermi(N, mu, f)

zero_statistics()
print "Cond: %f" % (hall_coeff_accurate(0.2))
print integrand_counter

# Plots

In [ ]:
#### Plot integrands and energies
mu = 0.2
N = calc_fock_N(mu)

import matplotlib as mpl
from matplotlib import pyplot
extent = pi
X = np.arange(-extent, extent, 0.06)
Y = np.arange(-extent, extent, 0.06)

fig = mpl.pyplot.figure()
zero_statistics()

def draw_Z(sp, Z, area):
    sp.imshow(Z, extent=(-extent, extent, -extent, extent))
    sp.contour(Z, extent=(-extent, extent, -extent, extent))

    # add the line
    sp.plot(X, [ min(area(x), extent) for x in X], "k-")
    sp.plot(X, [-min(area(x), extent) for x in X], "k-")

for m in range(3):

    area = m_fermi_area(N, m, mu)
    energy = m_energy(N, mu)
    #integrand = fock_element_integrand(N, mu, [N1_ELEM, N2_ELEM, (YZD, YZD, REAL)])
    integrand = fock_element_integrand(N, mu, [Na_ELEM, Nb_ELEM, N1_ELEM])
    
    # create the energy subplot
    Z = [[j for j in X] for i in Y]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Z[iy][ix] = energy(X[ix], Y[iy], [m])[0]

    energy_sp = fig.add_subplot(3, 4, 1+m*4)
    draw_Z(energy_sp, Z, area)
    if m == 0:
        energy_sp.set_title("Energy")
    
    # create the three graphs
    Zs = [np.zeros((len(X), len(Y))), np.zeros((len(X), len(Y))), np.zeros((len(X), len(Y)))]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Zs[0][iy][ix], Zs[1][iy][ix], Zs[2][iy][ix] = integrand(X[ix], Y[iy], [m])

    for j in range(3):
        integrand_sp = fig.add_subplot(3, 4, 1+m*4+1+j)
        draw_Z(integrand_sp, Zs[j], area)
        if m == 0:
            integrand_sp.set_title("integrand j=" + str(j))

        
mpl.pyplot.show(fig)

In [ ]:
#### view integrand

import matplotlib as mpl
from matplotlib import pyplot
X = np.arange(0, pi, 0.7)
Y = np.arange(0, pi, 0.7)

Z = [[j for j in X] for i in Y]
for ix in range(len(X)):
    for iy in range(len(Y)):
        Z[ix][iy] = m_integrand(guess_N1, guess_N2, guess_N3, mu)(X[ix], Y[iy], [0, 1, 2])[0]

print np.array_str(np.array(Z), precision=5)

In [ ]:
#### Recreate Energy Graphs
import matplotlib as mpl
from matplotlib import pyplot
xmax = 3.14
X = np.arange(-xmax, xmax, 0.01)

fig = mpl.pyplot.figure()
zero_statistics()
#mus = [-0.031, 0.034, 0.071]
mus = [0.14, 0.2, 0.22, 0.26]
print global_params_str()

for mu_i in range(len(mus)):
    mu = mus[mu_i]
    N = calc_fock_N(mu)
    guess_N1, guess_N2 = N[0], N[1]

    energy = m_energy(N, mu)
    integrand = fock_element_integrand(N, mu, [N1_ELEM, N2_ELEM, N2_ELEM])

    energy_sp = fig.add_subplot(1, len(mus), mu_i+1)

    for m in range(3):
        Y = [energy(x, 0, [m])[0] for x in X]
        energy_sp.plot(X, Y, ["r-", "b-", "g-"][m], linewidth = 4.0)
        energy_sp.set_autoscale_on(False)
        
    energy_sp.plot([-xmax, xmax], [mu, mu], "pc--", linewidth = 4.0)
    energy_sp.set_title("mu=%f" % mu)

mpl.pyplot.show(fig)

In [ ]:
guess_N1, guess_N2, guess_N3 = 0.00900515,  0.0039268, 0.0039268

In [ ]:
##### Show Statistics
import matplotlib as mpl
from matplotlib import pyplot

print "The integrand was called", integrand_counter, "times"
print "The energy was called",energy_counter, "times"
print "The integraion included ", bound_counter, "different x values"

dim = 400
Z = np.zeros((dim*2, dim*2))

for p in statistics:
    zx = p[0] * dim / pi
    zy = p[1] * dim / pi
    Z[int(zy)+dim, int(zx)+dim] += 10

pyplot.matshow(Z, extent=(0, pi, 0, pi))
pyplot.show()

In [ ]:
### Create the n's graphs for different Us
import matplotlib as mpl
from matplotlib import pyplot

class Graph(object):
    def __init__(self, solver, mus):
        self.solver = solver
        self.mus = mus
    
    def get_ns(self):
        orbit1, orbit2, orbit3 = [], [], []
        for mu in mus:
            o1, o2, o3 = self.solver.calc_n(mu)
            orbit1.append(o1)
            orbit2.append(o2)
            orbit3.append(o3)
        return orbit1, orbit2, orbit3
    
    def draw_graph(self, plt, style="-", width=4.0):
        orbit1, orbit2, orbit3 = self.get_ns()
        plt.plot(mus, orbit1, "r"+style, linewidth = width)
        plt.plot(mus, orbit2, "b"+style, linewidth = width)
        plt.plot(mus, orbit3, "g"+style, linewidth = width)
        plt.set_title(str(self.solver))
        
mus = [0 + i*0.02 for i in range(20)]

graphs = [
    #[Graph(Solver(2.0, 0.01, True) mus),  Graph(Solver(2.0, 0.01, False), mus)],
    #[Graph(Solver(1.3, 0.01, True) mus),  Graph(Solver(1.3, 0.01, False), mus)],
    #[Graph(Solver(2.0, 0.025, True) mus), Graph(Solver(2.0, 0.025, False), mus)],
    #[Graph(Solver(1.5, 0.025, True) mus), Graph(Solver(1.5, 0.025, False), mus)],
    #[Graph(Solver(2.7, 0.04, True) mus),  Graph(Solver(2.7, 0.04, False), mus)],
    [Graph(Solver(2.7, 0.04, True), mus)],
    #[Graph(1.9, 0.04, True, mus), Graph(1.9, 0.04, False, mus)],
]

for g in graphs:
    fig = mpl.pyplot.figure()
    for i,sub_g in enumerate(g):
        style = ["-", "--"][i]
        sp = fig.add_subplot(1, 1, 1)
        t = sp.get_title()
        sub_g.draw_graph(sp, style, 1.0)
        sp.set_title(sp.get_title() + "(" + style + ") " + t)
    fig.savefig("./graphs/compare_U%.1f_dso%.3f.png" % (g[0].solver.U, g[0].solver.dso))

mpl.pyplot.show()

In [ ]:
from scipy import optimize

def best_line_throuh_interval(intervals):
    intervals_x = sorted(intervals.keys())
    intervals_ys = [intervals[x] for x in intervals_x]
    intervals_len = np.array([i[1] - i[0] for i in intervals_ys])
    var_guess = [i[0] for i in intervals_ys] +  intervals_len/2
    
    def line_score(dots):
        dots_x = intervals_x
        dots_y = dots

        # calculate the score
        score = 0
        for i in range(len(crit_intervals)-1):
            score += (dots_y[i] - dots_y[i+1])**2 + (dots_x[i] - dots_x[i+1])**2
        return score
    
    ans = optimize.minimize(line_score, var_guess, bounds=intervals_ys)
    return np.array(intervals_x), np.array(ans["x"])

In [ ]:
# Plot the non mon area graphs

import matplotlib as mpl
from matplotlib import pyplot

def plot_non_mon_area(area_map, plt):
    # plot the dots
    red_U    = [x[0].pos[0] for x in area_map if not x[1]]
    red_dso  = [x[0].pos[1] for x in area_map if not x[1]]
    blue_U   = [x[0].pos[0] for x in area_map if x[1]]
    blue_dso = [x[0].pos[1] for x in area_map if x[1]]

    plt.plot(blue_dso, blue_U, "xb")
    plt.plot(red_dso, red_U, "xr")

    # find the critical intervals
    red_dots = [x[0] for x in area_map if x[1]]
    blue_dots = [x[0] for x in area_map if not x[1]]
    crit_intervals = {}
    for r in red_dots:

        # find the closet blue point
        same_dso = [x for x in blue_dots if x.pos[1] == r.pos[1] and x.pos[0] > r.pos[0]]
        sorted_points = sorted(same_dso, key=lambda x: x.pos[0] - r.pos[0])
        b = sorted_points[0]
        crit_intervals[r.pos[1]] = [r.pos[0], b.pos[0]]

    crit_dots = [[], []]
    crit_dots[0], crit_dots[1] = best_line_throuh_interval(crit_intervals)
    plt.plot(crit_dots[0], crit_dots[1])

    # understand the graph area
    dso_min = min([x[0].pos[1] for x in area_map])
    dso_max = max([x[0].pos[1] for x in area_map])
    U_min = min([x[0].pos[0] for x in area_map])
    U_max = max([x[0].pos[0] for x in area_map])
    graph_rect = [dso_min, dso_max, U_min - (U_max-U_min)*0.2, U_max + (U_max-U_min)*0.2]

    # fill the rect
    red_area = copy.deepcopy(crit_dots)
    green_area = copy.deepcopy(crit_dots)
    dsos = [x[0].pos[1] for x in area_map]
    red_area[0] = np.append(red_area[0], [max(dsos), min(dsos)])
    red_area[1] = np.append(red_area[1], [graph_rect[3], graph_rect[3]])
    green_area[0] = np.append(green_area[0], [max(dsos), min(dsos)])
    green_area[1] = np.append(green_area[1], [graph_rect[2], graph_rect[2]])

    plt.fill(red_area[0], red_area[1], color="r", alpha = 0.2)
    plt.fill(green_area[0], green_area[1], color="g", alpha = 0.2)

    plt.axis(graph_rect)
    plt.set_xlabel("dso [Ev]")
    plt.set_ylabel("U [Ev]")

    plt.grid(True)

fig = pyplot.figure()
hartree_sp = fig.add_subplot(1, 2, 1)
fock_sp = fig.add_subplot(1, 2, 2)
plot_non_mon_area(hartree_area_map, hartree_sp)
plot_non_mon_area(fock_area_map, fock_sp)
fock_sp.set_title("Fock")
hartree_sp.set_title("Hartree")
pyplot.show(fig)

In [ ]:
#### Plot conductivity integrand
mu = 0.1
N = calc_fock_N(mu)

import matplotlib as mpl
from matplotlib import pyplot
extent = pi
X = np.arange(-extent, extent, 0.06)
Y = np.arange(-extent, extent, 0.06)

fig = mpl.pyplot.figure()
zero_statistics()

def draw_Z(sp, Z, area):
    sp.imshow(Z, extent=(-extent, extent, -extent, extent))
    sp.contour(Z, extent=(-extent, extent, -extent, extent))

    # add the line
    sp.plot(X, [ min(area(x), extent) for x in X], "k-")
    sp.plot(X, [-min(area(x), extent) for x in X], "k-")

for m in range(3):

    area = m_fermi_area(N, m, mu)
    energy = m_energy(N, mu)
    integrand_xx = sigma_xx_integrand(N, mu)
    integrand_xy = sigma_xy_integrand(N, mu)
    
    # create the energy subplot
    Z = [[j for j in X] for i in Y]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Z[iy][ix] = energy(X[ix], Y[iy], [m])[0]

    energy_sp = fig.add_subplot(3, 4, 1+m*4)
    draw_Z(energy_sp, Z, area)
    if m == 0:
        energy_sp.set_title("Energy")
    
    # create the three graphs
    Zs = [np.zeros((len(X), len(Y))), np.zeros((len(X), len(Y)))]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Zs[0][iy][ix] = integrand_xx(X[ix], Y[iy])[m*2]
            Zs[1][iy][ix] = integrand_xy(X[ix], Y[iy])[m*2]

    for j in range(2):
        integrand_sp = fig.add_subplot(3, 3, 1+m*3+1+j)
        draw_Z(integrand_sp, Zs[j], area)
        if m == 0:
            integrand_sp.set_title("integrand j=" + str(j))

        
mpl.pyplot.show(fig)

In [ ]:
### plot the hall coeffecient
mus = [0 + i*0.02 for i in range(20)]
conductivity = []
hall_coeff = []
for mu in mus:
    zero_statistics()
    conductivity += [sigma_xx(mu)]
    hall_coeff += [sigma_xy(mu)]

pyplot.subplot(2, 1, 1)
pyplot.plot(mus, conductivity)
pyplot.title("Conductivity")
pyplot.subplot(2, 1, 2)
pyplot.plot(mus, hall_coeff)
pyplot.title("Hall Coefficient")
pyplot.show()

# Density Of States

In [ ]:

class DerivSolver(object):
    
    def __init__(self, U, dso, fock, dmu):
        self.solver = Solver(U, dso, fock) 
        self.dmu = dmu
    
    def calc_n(self, mu):
        return (self.solver.calc_n(mu) - slef.solver.calc_n(mu + self.dmu))/self.dmu
    
    def __str__(self):
        return "Deriv " + str(self.solver)

class PartialDerivSolver(object):
    def __init__(self, U, dso, fock, dmu):
        self.solver = Solver(U, dso, fock) 
        self.dmu = dmu
    
    def calc_n(self, mu):
        N = self.solver.calc_N(mu)
        return (calc_n(mu, N) - calc_n(mu + self.dmu, N))/self.dmu

    def __str__(self):
        return "Deriv " + str(self.solver)

mus = [0 + i*0.02 for i in range(20)]

graphs = [
    [Graph(PartialDerivSolver(2.7, 0.04, True, 0.1), mus), Graph(PartialDerivSolver(2.7, 0.04, False, 0.1), mus)],
]

for g in graphs:
    fig = mpl.pyplot.figure()
    for i,sub_g in enumerate(g):
        style = ["-", "--"][i]
        sp = fig.add_subplot(1, 1, 1)
        t = sp.get_title()
        sub_g.draw_graph(sp, style, 1.0)
        sp.set_title(sp.get_title() + "(" + style + ") " + t)
    #fig.savefig("./graphs/deriv_compare_U%.1f_dso%.3f.png" % (g[0].solver.U, g[0].solver.dso))

mpl.pyplot.show()
